# Stream modes

LangGraph支持多种流式模式。主要有如下：
- `values`: 这种模式下会返回历史的Messages记录；
- `updates`: 这种模式下只返回当前最新的Messages。

# Stream full state

In [2]:
from typing import Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

model = ChatOpenAI(temperature=0)
graph = create_react_agent(model, tools)

In [8]:
inputs = {"messages": [("human", "what's the weather in sf")]}
async for chunk in graph.astream(inputs, stream_mode="values"):
    # chunk["messages"][-1].pretty_print()
    print(chunk)

{'messages': [HumanMessage(content="what's the weather in sf", id='38d7d861-5882-4649-b137-0f1c364ae84a')]}
{'messages': [HumanMessage(content="what's the weather in sf", id='38d7d861-5882-4649-b137-0f1c364ae84a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1j4koz2Pe0ImqZrFqiaX2Q6b', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c832e4513b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8ab7fd69-e767-49ba-9dd5-2b099518abf9-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_1j4koz2Pe0ImqZrFqiaX2Q6b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 14, 'total_tokens': 71})]}
{'messages': [HumanMessage(content="what's the weather in sf", id='38d7d861-5882-4649-b137-0f1c364ae84a'), AIMessage(content='', addi

## 只获取最终结果
如果只想获取最终的结果，可以使用如下形式：

In [4]:
inputs = {"messages": [("human", "what's the weather in sf")]}
async for chunk in graph.astream(inputs, stream_mode="values"):
    final_result = chunk

In [5]:
final_result

{'messages': [HumanMessage(content="what's the weather in sf", id='bfbba8ba-fbe5-4b11-8aa0-86b1973c7f06'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mByEhXUAB8NkCnuwkgfGpzBr', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d623d511-99c1-416f-84b7-b78a46873025-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_mByEhXUAB8NkCnuwkgfGpzBr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 14, 'total_tokens': 71}),
  ToolMessage(content="It's always sunny in sf", name='get_weather', id='c4fb2df8-6b2e-4d6f-8a82-073d6c14a9e3', tool_call_id='call_mByEhXUAB8NkCnuwkgfGpzBr'),
  AIMessage(content='The weather in San Francisco is currently sunny.', response_

# Stream state updates of your graph

In [7]:
inputs = {"messages": [("human", "what's the weather in sf")]}
async for chunk in graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values)
        print("\n\n")

Receiving update from node: 'agent'
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_e3w46QemI5PiVia3qVc4I3uE', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c832e4513b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-791e495b-e87a-4b3c-a982-e0ee1a41aa95-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_e3w46QemI5PiVia3qVc4I3uE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 14, 'total_tokens': 71})]}



Receiving update from node: 'tools'
{'messages': [ToolMessage(content="It's always sunny in sf", name='get_weather', tool_call_id='call_e3w46QemI5PiVia3qVc4I3uE')]}



Receiving update from node: 'agent'
{'messages': [AIMessage(content='The weather in San Francisco is currently sunny.', respons